# Sliding window PCA on malaria gen data

This notebook computes sliding window PCAs for data accessible through the `malariagen_data` package. After running this notebook, you can visualise the results with the `plot_sliding_window_pca.ipynb` notebook. The computations take a while (approx. 25 minutes for chrom 3RL (84Mb) on 620 samples using dask cluster with 50 nodes with default parameters for variant selection) so it is advisable to run it on a cluster (here we use dask). The output is approx. 4 Mb for PC1 and 2 on chrom 3RL for 620 samples.

You need to create an account to access the data, more info here: https://malariagen.github.io/vector-data/vobs/vobs-data-access.html  
Then you need to sign in once per session, by running `gcloud auth application-default login` in the same environment used to run this notebook and following the instructions.

### Imports

In [1]:
import malariagen_data
import pandas as pd
import numpy as np
import os
import allel

from dask.distributed import Client
import dask
dask.config.set(**{'array.slicing.split_large_chunks': False}) # Silence large chunk warnings
import dask.array as da
from dask import delayed, compute
from dask_gateway import Gateway
import numba
import warnings
warnings.filterwarnings("ignore", message="Sending large graph of size") #Silence large graph warnings

### Parameters
NOTE: the following changes have to be made to run this on Ag instead of Af. 
- Change `Af1` to `Ag3` in the access data function
- Remove the `site_filter_analysis` argument to use the default for Ag3
- Remove the steps used to add extra metadata
- Change `variant_filter_pass_funestus` in the helper function `prepare_genotypes_positions` to the desired filter (e.g. `variant_filter_pass_gamb_colu_arab` for all three vector species in the complex)

Additionally specify which samples you want to use, based on the metadata, see below

### Access data

In [3]:
#Here we read in data for funestus using the static cutoff site filter
af1 = malariagen_data.Af1(site_filters_analysis="sc_20220908")
af1

<MalariaGEN Af1 API client>
Storage URL             : gs://vo_afun_release_master_us_central1
Data releases available : 1.0
Results cache           : None
Cohorts analysis        : 20240515
Site filters analysis   : sc_20220908
Software version        : malariagen_data 10.0.0
Client location         : Iowa, United States (Google Cloud us-central1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v10.0.0/Af1.html

In [4]:
#we read in the sample metadata for Af1.0 and
#we add some of the metadata contained in supplementary table 1 tab 2
extra_meta = pd.read_csv("../../metadata/supp1_tab2.csv")
extra_meta.rename(columns={'VBS_sample_id': 'sample_id'}, inplace=True)
#subset to useful columns and prevent duplicates
extra_meta = extra_meta[['sample_id',
       'geographic_cohort', 'geographic_cohort_colour', 'geographic_cohort_code'
       'geographic_cohort_shape', 'PCA_cohort', 'PCA_cohort_colour',
       'karyotype_3La', 'karyotype_3Ra', 'karyotype_3Rb', 'karyotype_2Ra',
       'karyotype_2Rh', 'karyotype_2Rt', 'mitochondrial_id', 'subset_1',
       'subset_2', 'subset_3']]
af1.add_extra_metadata(extra_meta)
meta = af1.sample_metadata(sample_sets='1.0')
meta.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,karyotype_3La,karyotype_3Ra,karyotype_3Rb,karyotype_2Ra,karyotype_2Rh,karyotype_2Rt,mitochondrial_id,subset_1,subset_2,subset_3
0,VBS24195,1229-GH-A-GH01,Samuel Dadzie,Ghana,Dimabi,2017,8,9.420,-1.083,F,...,NaN,3R+/a,3Rb/b,2Ra/a,2R+/+,2R+/+,funestus-lineageI-clusterB,Y,Y,Y
1,VBS24196,1229-GH-A-GH02,Samuel Dadzie,Ghana,Gbullung,2017,7,9.488,-1.009,F,...,NaN,3Ra/a,3Rb/b,2Ra/a,2R+/+,2R+/+,funestus-lineageI-clusterB,Y,Y,Y
2,VBS24197,1229-GH-A-GH03,Samuel Dadzie,Ghana,Dimabi,2017,7,9.420,-1.083,F,...,NaN,3R+/a,3Rb/b,2Ra/a,2R+/+,2R+/+,funestus-lineageI-clusterB,Y,Y,Y
3,VBS24198,1229-GH-A-GH04,Samuel Dadzie,Ghana,Dimabi,2017,8,9.420,-1.083,F,...,NaN,3R+/a,3Rb/b,2Ra/a,2R+/+,2R+/+,funestus-lineageI-clusterB,Y,Y,Y
4,VBS24199,1229-GH-A-GH05,Samuel Dadzie,Ghana,Gupanarigu,2017,8,9.497,-0.952,F,...,NaN,3Ra/a,3Rb/b,2Ra/a,2R+/+,2R+/+,funestus-lineageI-clusterB,Y,Y,Y


In [5]:
#Run on all samples except the Ghana_Northern-Region geographic cohort
sample_idx = meta.loc[meta.geographic_cohort!='Ghana_Northern-Region'].index
len(sample_idx)

620

### Set up dask cluster

In [24]:
gateway = Gateway()
conda_prefix = os.environ["CONDA_PREFIX"]
current_environment = 'global/'+conda_prefix.split('/')[5]
cluster = gateway.new_cluster(
    profile='standard', 
    conda_environment = current_environment,
)
cluster

In [25]:
client=cluster.get_client()

In [26]:
cluster.scale(50)

### Helper functions

In [9]:
def prepare_genotypes_positions(chrom, posl, posu, release, sample_idx):
    #access snp_calls xarray dataset
    snp_calls = af1.snp_calls(region=f'{chrom}:{posl}-{posu}', sample_sets=release)
    variant_filter = snp_calls.variant_filter_pass_funestus.compute()
    #apply site filter to genotypes
    gt = snp_calls.call_genotype[variant_filter,:,:]
    #select the desired samples
    gt = gt[:,sample_idx,:]
    #broadcast gt as genotype dask array
    gt = allel.GenotypeDaskArray(gt.data)
    #apply site filter to positions
    pos = snp_calls.variant_position[variant_filter]
    #broadcast positions as array
    pos = pos.data
    return(gt, pos)

In [10]:
# Define a function to count the presence of each allele in a given array of genotypes (samples * variants * alleles)
# and return an array of allele counts with a row for each possible allele (limited by max_allele) for each sample, and column for each variant
@numba.njit(
    numba.int8[:, :](numba.int8[:, :, :], numba.int8), nogil=True)
def numpy_genotype_tensor_to_allele_counts_melt(gt, max_allele):
    # Create an array of zeros (for defaults) with the same number of colums (variants) as the genotype array but a row for each allele, for each sample
    out = np.zeros((gt.shape[0] * (max_allele + 1), gt.shape[1]), dtype=np.int8)
    # For each row (sample) in the genotype array
    for i in range(gt.shape[0]):
        # For each column (variant) in the genotype array
        for j in range(gt.shape[1]):
            # For each allele in the genotype array 
            for k in range(gt.shape[2]):
                allele = gt[i, j, k]
                # If the value in the genotype array at this row and colum and 3rd dimension (i.e. the allele value) is between 0 and max_allele  
                if 0 <= allele <= max_allele:
                    # Increment the value of the `out` array at the row corresponding to this allele for this sample, at the corresponding variant column
                    out[(i * (max_allele + 1)) + allele, j] += 1
    return out

In [11]:
# Define a function to do the same thing as the previous function
# but the Dask way
def dask_genotype_tensor_to_allele_counts_melt(gt, max_allele):

    # Determine output chunks - change axis 0; preserve axis 1; drop axis 2.
    dim0_chunks = tuple(np.array(gt.chunks[0]) * (max_allele + 1))
    chunks = (dim0_chunks, gt.chunks[1])

    return gt.map_blocks(
        numpy_genotype_tensor_to_allele_counts_melt,
        max_allele=max_allele,
        chunks=chunks,
        dtype="i1",
        drop_axis=2,
    )

In [19]:
def select_variants(gt, pos, n_samples, maf, missing_frac, variant_density, 
                    af_tolerance, frac_acc, seed = 42):
    
    #count alleles 
    ac = gt.count_alleles(max_allele=3)
    #compute frequencies
    af = ac.to_frequencies()
    
    print(f"Read in {gt.shape[0]} accessible sites")
    
    #filters for missingness, near-biallelism and maf
    missing_filter = ac.sum(axis=1) >= (1-missing_frac)*2*n_samples
    #Find nearly biallelic sites: 
    #Require two alleles with frequencies ≥ maf and
    #tolerate more than two alleles if the others are below the af_tolerance
    maf_filter = ((af >= maf).sum(axis=1) == 2) & ((af <= af_tolerance).sum(axis=1) == 2)
    
    #If many variants, downsample to the desired number
    #Variant density is given in the number of desired variants Mb 
    #(taking into account the mean accessibility rate)
    combined_filter = ((missing_filter) & (maf_filter)).compute()
    n_candidates = combined_filter.sum()
    print(f"Contains {n_candidates} candiate variant sites")
    n_variants = int(variant_density * gt.shape[0] / frac_acc / 1_000_000)
    print(f"We want {n_variants} sites for sliding window PCA")
    if n_candidates > n_variants:
        candidate_idx = np.arange(gt.shape[0])[combined_filter]
        rng = np.random.default_rng(seed=seed)
        selection_idx = rng.choice(candidate_idx, n_variants, replace=False)
        selection_idx = np.sort(selection_idx)
    else:
        selection_idx = np.arange(gt.shape[0])[combined_filter]
    
    #get filtered allele counts
    ac_f = ac.take(selection_idx, axis=0)
    pos_f = pos[selection_idx]
    gt_f = gt.take(selection_idx, axis=0)
    
    #get major allele index
    major_alleles = da.argmax(ac_f, axis=1)
    
    #melted allele counts
    melted_counts = dask_genotype_tensor_to_allele_counts_melt(gt_f, max_allele=3)
    variant_counts = da.take(melted_counts, np.arange(major_alleles.shape[0])*4+major_alleles.compute(), axis=0)
    variant_pos = pos_f.compute()
    
    return variant_counts, variant_pos

In [13]:
def compute_windows(n_vars, window_size, step_size):
    n_windows = (n_vars-window_size+step_size)//step_size
    starts = (np.arange(0,n_windows)*step_size).astype(int)
    ends = (starts+window_size).astype(int)
    return(starts,ends,n_windows)

In [14]:
def variant_check(computed_variants):
    for i in range(3):
        if (computed_variants == i).all(axis=1).any():
            loc_invariant = (computed_variants == i).all(axis=1)
            computed_variants = computed_variants[~loc_invariant]
            print(f'Removed {sum(loc_invariant)} invariant sites')
    return(computed_variants)

In [15]:
def perform_PCA_in_window(computed_variants, n_pc):
    coords, model = allel.pca(computed_variants, n_components=n_pc, scaler='patterson')
    explained_variance = model.explained_variance_ratio_
    return(coords, explained_variance)

In [23]:
def process_reading_frame(
        chrom, sample_idx, n_pc, posl, posu, release, window_size, step_size, 
        maf, missing_frac, variant_density, af_tolerance, frac_acc):
    
    #read in genotypes and positions and subset to desired individuals
    gt, pos = prepare_genotypes_positions(
        chrom, posl, posu, release, sample_idx)
    
    #select variants and return counts per individual of variant allele
    variant_counts, variant_pos = select_variants(
        gt, pos, len(sample_idx), maf, missing_frac, 
        variant_density, af_tolerance, frac_acc)
    
    #split data into windows
    start_idx, end_idx, n_windows = compute_windows(
        len(variant_pos), window_size, step_size)
    
    #check that size of data is sufficient for at least one window
    if n_windows<1:
        return np.array([[[]]]), np.array([[]]), np.array([[]]), variant_pos.max()+step_size
    
    #generate objects to store results 
    pc_values = np.zeros((n_pc, n_windows, len(sample_idx)))
    var_exp = np.zeros((n_pc, n_windows))
    pos_win = np.zeros((n_windows, 3))
    
    #loop through windows to compute PCA
    for n, start, end in zip(np.arange(n_windows), start_idx, end_idx):
        variants_window = variant_counts[start:end].compute()
        variants_window = variant_check(variants_window)
        coords, variance = perform_PCA_in_window(variants_window, n_pc)
        
        for pc in np.arange(n_pc):
            #Check if you want to flip the PC axis
            if n>0:
                no_flip = np.sum(np.abs(pc_values[pc,n-1,:] - coords[:,pc]))
                flip = np.sum(np.abs(pc_values[pc,n-1,:] + coords[:,pc]))
                if flip < no_flip:
                    coords[:,pc] *= -1
            #store values for each pc
            pc_values[pc,n,:] = coords[:,pc]
        #store explained variance
        var_exp[:,n] = variance
        #store positions
        pos_win[n,:] = (variant_pos[start], variant_pos[end-1], variant_pos[start:end].mean())
        
    print(f'Computed PC in {n_windows} windows, spanning from {variant_pos[start_idx[0]]} to {variant_pos[end-1]}')
    return pc_values, var_exp, pos_win, variant_pos[start+step_size]

### Main function

In [20]:

def loop_through_reading_frames(chrom, sample_idx, outdir=None, n_pc=1, name=None, 
                                posl=-1, posu=-1, release='1.0',
                                reading_frame_size=10_000_000,
                                window_size = 5000, step_size=1000, maf=0.02, missing_frac=0.10, 
                                variant_density=5000, af_tolerance=0.001, frac_acc=0.77):
    
    """
    Top function to run the sliding window PCA
    Saves pc_values, explained_variance and positions as np files in outdir

    ARGUMENTS
    chrom: chromosome name
    sample_idx: sample indices in gt array
    outdir: directory where to save the data
    n_pc: number of pcs to compute (tested for 1 and 2)
    posl: lower genomic coordinate if not using the whole chromosome
    posu: upper genomic coordinate if not using the whole chromosome
    release: data release(s) to use
    reading_frame_size: number of bases to process at a time
    window_size: number of variant sites per window
    step_size: number of variants by which the window slides
    maf: minor allele frequency for selecting variants
    variant_density: desired mean number of variants per Mb
    af_tolerance: tolerated frequency of alternative minor alleles in selecting variants
    frac_acc: fraction of accessible sites (depending on site filter)
    """
    
    #If no name provided, use chrom as name
    if name is None:
        name = chrom
    #If no outdir provided, use `output`
    if outdir is None:
        outdir = 'output'
    #set up directory if it does not exist yet
    if not os.path.isdir(outdir):
        ! mkdir {outdir}
    
    #process to the end of the specified region or to the end of chromosome if not specified
    if posu==-1:
        posu = af1.genome_sequence(region=chrom).shape[0]
    #start from posl if specified, otherwise from start of chromosome
    rf_start = max(posl, 1)
    #process to the end of the specified region, or for the lenght of one reading frame
    rf_end = min(posu, rf_start+reading_frame_size)
    
    #compute pcs for the first reading frame
    pc_values, var_exp, pos_win, rf_start = process_reading_frame(
        chrom, sample_idx, n_pc, rf_start, rf_end, release, 
        window_size, step_size, maf, missing_frac, variant_density, 
        af_tolerance, frac_acc)
    
    #save intermediate results
    np.save(f'{outdir}/{name}_pc_values.npy', pc_values)
    np.save(f'{outdir}/{name}_var_exp.npy', var_exp)
    np.save(f'{outdir}/{name}_positions.npy', pos_win)    
    print(f'Processed chromosome {chrom} up to {int(pos_win[-1,1])}')
    
    #process further reading frames until the end of the region
    while rf_end < posu:
        rf_end = min(posu, rf_start+reading_frame_size)
        pc_values_rf, var_exp_rf, pos_win_rf, rf_start = process_reading_frame(
            chrom, sample_idx, n_pc, rf_start, rf_end, release, window_size, 
            step_size, maf, missing_frac, variant_density, af_tolerance, frac_acc)
        
        #Check that the latest reading frame is not returned empty
        if pc_values_rf.shape[1]>0:
            #Check if you want to flip the PC axes
            for pc in np.arange(n_pc):
                no_flip = np.sum(np.abs(pc_values_rf[pc,0,:] - pc_values[pc,-1,:]))
                flip = np.sum(np.abs(pc_values_rf[pc,0,:] + pc_values[pc,-1,:]))
                if flip < no_flip:
                    pc_values_rf[pc] *= -1
            
            #append data from latest reading frame
            pc_values = np.concatenate((pc_values, pc_values_rf),axis=1)
            var_exp = np.concatenate((var_exp, var_exp_rf), axis=1)
            pos_win = np.concatenate((pos_win, pos_win_rf))
            
        #save results
        np.save(f'{outdir}/{name}_pc_values.npy', pc_values)
        np.save(f'{outdir}/{name}_var_exp.npy', var_exp)
        np.save(f'{outdir}/{name}_positions.npy', pos_win)  
        print(f'Processed chromosome {chrom} up to {int(pos_win[-1,1])}')
        
        

### Run computation

In [30]:
#set parameters
chrom='3RL'
n_pc=2

In [31]:
loop_through_reading_frames(chrom=chrom, sample_idx=sample_idx, n_pc=n_pc)

Read in 8125579 accessible sites
Contains 478477 candiate variant sites
We want 52763 sites for sliding window PCA
Computed PC in 48 windows, spanning from 210065 to 9891797
Processed chromosome 3RL up to 9891797
Read in 8341156 accessible sites 
Contains 516570 candiate variant sites
We want 54163 sites for sliding window PCA
Computed PC in 50 windows, spanning from 9206548 to 19177477
Processed chromosome 3RL up to 19177477
Read in 7910978 accessible sites 
Contains 490313 candiate variant sites
We want 51369 sites for sliding window PCA
Computed PC in 47 windows, spanning from 18432665 to 28331822
Processed chromosome 3RL up to 28331822
Read in 8218631 accessible sites 
Contains 503788 candiate variant sites
We want 53367 sites for sliding window PCA
Computed PC in 49 windows, spanning from 27605384 to 37533738
Processed chromosome 3RL up to 37533738
Read in 7678500 accessible sites 
Contains 363082 candiate variant sites
We want 49860 sites for sliding window PCA
Computed PC in 45 

2024-10-25 10:20:35,575 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling (('getitem-b17d9653c4e5f435615d135b11c361da', 4, 3),)


Computed PC in 48 windows, spanning from 72837372 to 82780763
Processed chromosome 3RL up to 82780763
Read in 2346367 accessible sites 
Contains 105883 candiate variant sites
We want 15236 sites for sliding window PCA
Computed PC in 11 windows, spanning from 81993621 to 84433783
Processed chromosome 3RL up to 84433783


In [32]:
cluster.shutdown()